In [1]:
import cx_Oracle as cxo

In [3]:
conn = cxo.connect("Jackpot/jackpot@orcl")

In [5]:
cur = conn.cursor()

In [35]:
cur.execute("SELECT * FROM 일별주가 WHERE 기준일자 = (SELECT MAX(기준일자) FROM 일별주가)")
row = cur.fetchone()
print(row)

('20181128', 'A000020', 9480, 9490, 9150, 9320, -110, 138261, 2374964, 668, 8.5, -1.17, '5', 128592, 0, 10629, 1638)


In [17]:
res = cur.fetchmany(numRows=3)
print(res)

[('20181128', 'A000030', 15900, 16000, 15450, 15700, -200, 1451132, 187542308, -388973, 27.740000000000002, -1.26, '5', 2265040, 0, -301103, -92837), ('20181128', 'A000040', 512, 519, 507, 512, -6, 89990, 30711103, -644, 23.81, -1.16, '5', 4599, 0, -644, 18), ('20181128', 'A000050', 11250, 11250, 11050, 11150, -50, 10741, 623721, 490, 2.2800000000000002, -0.45, '5', 11958, 0, 490, -8)]


# DB to Numpy
## Numpy to CSV File

In [55]:
import numpy as np
import pandas as pd

In [56]:
cur.execute("SELECT * FROM 일별주가 WHERE 종목코드 = 'A008560' ORDER BY 기준일자 DESC")
row = cur.fetchone()
print(row)

('20181128', 'A008560', 4410, 4440, 4340, 4365, -35, 1197498, 48231530, 249047, 7.96, -0.8, '5', 523194, 0, 286295, -556886)


In [63]:
sql = """
SELECT A.시가,A.고가,A.저가,A.종가,A.전일대비,A.거래량,A.거래대금,A.등락률,A.개인거래대금,A.외인거래대금,A.기관거래대금
     , B.시가,B.고가,B.저가,B.종가,B.전일대비,B.거래량,B.외인보유,B.외인보유전일대비,B.외인비중,B.등락률,B.거래대금
     , C.개인순매수,C.외인순매수,C.기관순매수,C.금융투자,C.보험,C.투신,C.은행,C.기타금융,C.연기금,C.기타법인,C.기타외인,C.사모펀드,C.국가지자체
     , LEAD(B.등락률) over (order by A.기준일자 ASC) CLASS
  FROM 일별지수 A
     , 일별주가 B
     , 일별수급 C
 WHERE 1=1
   AND A.기준일자 = B.기준일자
   AND A.기준일자 = C.기준일자
   AND B.종목코드 = C.종목코드
   AND A.지수코드 = 'U001' --DECODE(:시장구분, 1, 'U001', 'U201')
   --AND B.기준일자 = '20180619'
   AND B.종목코드 = 'A008560' --:종목코드
 ORDER BY A.기준일자 DESC
"""

In [64]:
cur.execute(sql)

<cx_Oracle.Cursor on <cx_Oracle.Connection to Jackpot@orcl>>

In [65]:
alldata = cur.fetchall()

In [66]:
#npdata = np.array(alldata)
#npdata.ndim

df = pd.DataFrame(alldata)
df.to_csv("D:\Python\keras\stock.csv", header=False, index=False)

In [ ]:
cur.close()
conn.close()